In [15]:
import os
import numpy as np
import pandas as pd
import glob
import datetime
from datetime import datetime, date, timedelta

#carregando o df com tipo de petrecho por mmsi
df_apoio = pd.read_csv('C:/Users/mathe/mestrado/fishing_vessels_v20190520_fishing_vessels_v20190702.csv')

#convertendo formato do mmsi para numero
df_apoio['mmsi'] = df_apoio['mmsi'].astype(np.int32)

#carregando o arquivo de referência para as analises gfw_real
ref = pd.read_csv('C:/Users/mathe/mestrado/lista_arcgis.csv')

#onde a pasta está localizada
file_path = 'C:/Users/mathe/mestrado/daily_csv'

#não entendi muito bem, talvez seja acessar os csvs'
read_files = glob.glob(os.path.join(file_path, '*.csv'))

#criar uma lista em q cada item é um csv
np_array_values = []

#adicionar itens da pasta à lista
for files in read_files:
    dia_naveg = pd.read_csv(files, header=0)
    np_array_values.append(dia_naveg)

#Agrupar todos os csvs' da lista num df só
frame = pd.concat(np_array_values, axis = 0, ignore_index = True)


In [16]:
df = frame.query('mmsi in @ref.mmsi').copy()

In [17]:
df.mmsi.unique()

array([770001221, 710289219, 710500000, 710100200, 710004421, 710131976,
       710240366, 710232550, 710000134, 710008290, 710001955, 710000799,
       999792275, 710792275], dtype=int64)

In [18]:
frame.head()

,date,lat_bin,lon_bin,mmsi,fishing_hours
0,2012-01-01 00:00:00 UTC,-461,-607,416755000,2.457917
1,2012-01-01 00:00:00 UTC,-461,-607,440548000,2.400417
2,2012-01-01 00:00:00 UTC,-460,-607,440548000,1.614444
3,2012-01-01 00:00:00 UTC,-460,-607,416032500,11.497083
4,2012-01-01 00:00:00 UTC,-459,-607,416842000,11.521944


In [19]:
#filtrar dados proximos ao brasil
#df = frame.query(' -370 < lat_bin &  lat_bin < 92 ').query(' -600 < lon_bin & lon_bin < -250').copy()
df = df.query('( -500 <= lat_bin &  lat_bin <= 140) & (-700 <= lon_bin & lon_bin <= -180)').copy()
#df = df.query('( -500 <= lat_bin <= 140) & (-700 <= lon_bin <= -180)').copy()

#dividr as coordenadas pra ficar padrão
df['lat_bin'] = df['lat_bin']/10
df['lon_bin'] = df['lon_bin']/10

#adicionando geartype de cada embarcação

df2 = df.merge(df_apoio, on='mmsi', sort=False )

#colocar date no formato datetime
df2['date'] = pd.to_datetime(df2['date'])

#sort by date
df2.sort_values(['mmsi', 'date'], ascending=[True, False])

#colocar previous latitude e longitude
#essa prineira linha tem apenas as condições: se o time delta for menor que 6 dias e o mmsi for igual à linha anterior
df2['lat_prev'] = np.where((df2['date'] - df2['date'].shift(1) < timedelta(days= 6)) & (df2['mmsi'] == df2['mmsi'].shift(1)),
                          df2['lat_bin'].shift(1), df2['lat_bin']) #o que faz tá aqui, sendo true e false separados por vírgulas


df2['lon_prev'] = np.where((df2['date'] - df2['date'].shift(1) < timedelta(days= 6)) & (df2['mmsi'] == df2['mmsi'].shift(1)),
                          df2['lon_bin'].shift(1), df2['lon_bin'])        

In [20]:
#fechar o que precisa pro gfw_estimado

#crianr colunas de month e week
df2['week'] = df2['date'].dt.week
df2['month'] = df2['date'].dt.month

#salvando todos os anos
#df.to_csv('C:/Users/mathe/mestrado/gfw_est_all_years.csv', index= False)

#pegar 2018
df3 = df2[df2['date'].dt.year == 2018]

#salvando só 2018
#df.to_csv('C:/Users/mathe/mestrado/gfw_est.csv', index= False)

In [21]:
#fechar o que precisa pro gfw_real

#limitar para as embarcações presentes no rgp
#df2 = df.query('mmsi in @ref.mmsi').copy()

#colocar os dados reais obtidos pelo rgp
df4 = df3.merge(ref[['mmsi','petrecho', 'Comprimento', 'AB']], on='mmsi')

#salvando
#df3.to_csv('C:/Users/mathe/mestrado/gfw_real_final2.csv', index= False, encoding='latin-1')

In [22]:
df.mmsi.unique()

array([770001221, 710289219, 710500000, 710100200, 710004421, 710131976,
       710240366, 710232550, 710000134, 710008290, 710001955, 710000799,
       999792275, 710792275], dtype=int64)

In [23]:
df2.mmsi.unique()

array([770001221, 710289219, 710500000, 710100200, 710004421, 710131976,
       710240366, 710232550, 710000134, 710008290, 710001955, 710000799,
       999792275, 710792275], dtype=int64)

In [24]:
df3.mmsi.unique()

array([710289219, 710500000, 710004421, 710131976, 710240366, 710232550,
       710000134, 710008290, 710001955, 710000799, 999792275, 710792275],
      dtype=int64)

In [25]:
df2['year'] = df2['date'].dt.year
df5 = df2.query('year == 2018').copy()
df5.mmsi.unique()

array([710289219, 710500000, 710004421, 710131976, 710240366, 710232550,
       710000134, 710008290, 710001955, 710000799, 999792275, 710792275],
      dtype=int64)

In [26]:
df2.query('mmsi == 770001221').copy()

,date,lat_bin,lon_bin,mmsi,fishing_hours,flag,vessel_class,registry_vessel_class,inferred_vessel_class,inferred_vessel_class_score,...,active_2014,active_2015,active_2016,active_2017,active_2018,lat_prev,lon_prev,week,month,year
0,2015-06-04,-30.0,-48.2,770001221,6.012222,BRA,fishing,fishing,pole_and_line,0.471516,...,False,True,True,True,True,-30.0,-48.2,23,6,2015
1,2015-06-04,-29.9,-48.2,770001221,5.787500,BRA,fishing,fishing,pole_and_line,0.471516,...,False,True,True,True,True,-30.0,-48.2,23,6,2015
2,2015-06-04,-29.8,-48.2,770001221,4.346389,BRA,fishing,fishing,pole_and_line,0.471516,...,False,True,True,True,True,-29.9,-48.2,23,6,2015
3,2015-06-04,-29.7,-48.2,770001221,4.308333,BRA,fishing,fishing,pole_and_line,0.471516,...,False,True,True,True,True,-29.8,-48.2,23,6,2015
4,2015-06-05,-30.5,-48.2,770001221,9.883056,BRA,fishing,fishing,pole_and_line,0.471516,...,False,True,True,True,True,-29.7,-48.2,23,6,2015
5,2015-06-05,-30.5,-48.1,770001221,3.997778,BRA,fishing,fishing,pole_and_line,0.471516,...,False,True,True,True,True,-30.5,-48.2,23,6,2015
6,2015-06-05,-30.4,-48.1,770001221,4.910000,BRA,fishing,fishing,pole_and_line,0.471516,...,False,True,True,True,True,-30.5,-48.1,23,6,2015
7,2015-06-05,-30.2,-48.2,770001221,2.603333,BRA,fishing,fishing,pole_and_line,0.471516,...,False,True,True,True,True,-30.4,-48.1,23,6,2015
8,2015-06-05,-30.0,-48.2,770001221,5.621250,BRA,fishing,fishing,pole_and_line,0.471516,...,False,True,True,True,True,-30.2,-48.2,23,6,2015
9,2015-06-06,-31.5,-49.8,770001221,0.779444,BRA,fishing,fishing,pole_and_line,0.471516,...,False,True,True,True,True,-30.0,-48.2,23,6,2015
